In [1]:
import boto3
import pandas as pd
from io import StringIO

# Define your bucket name and file key (file path in S3)
BUCKET_NAME = "blue-blood-data"
FILE_KEY = "BigQuery Blue Blood DB Data.csv"  # Change to your actual file path in S3

# Create an S3 client
s3 = boto3.client("s3")

# Fetch the file from S3
response = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)

# Read the CSV file into a pandas DataFrame
csv_content = response["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(csv_content))

# Print DataFrame
df.head()

,subject_id,prescription_start,prescription_drug,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,Phenylephrine HCl,60,mg,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10013,2125-10-05T00:00:00,Vasopressin,100,UNIT,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10013,2125-10-05T00:00:00,Iso-Osmotic Dextrose,50,ml,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10013,2125-10-05T00:00:00,Dobutamine,250,mg,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10013,2125-10-05T00:00:00,Aspirin,325,mg,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import pandas as pd
import numpy as np

# Print values of columns that start with 'pre_' or 'post_'
'''for col in df.columns:
    if col.startswith("pre_") or col.startswith("post_"):
        print(f"Column: {col}")
        print(df[col])'''
for col in df.select_dtypes(include=[np.number]).columns:
    if (df[col] < 0).any():
        print(f"Column '{col}' contains negative values.", df[col].values.min())

Column 'pre_baseexcess' contains negative values. -16.0
Column 'post_baseexcess' contains negative values. -17.0


In [3]:
# convert prescription_dose_val_rx to numeric
print(df['prescription_dose_val_rx'].dtype)
df['prescription_dose_val_rx'] = pd.to_numeric(df['prescription_dose_val_rx'], errors='coerce')
print(df['prescription_dose_val_rx'].dtype)

object
float64


In [4]:
# get numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
numeric_cols = numeric_cols[1:]  # exclude the first column which is the patient_id

# fill null values in numeric columns with -200
df[numeric_cols] = df[numeric_cols].fillna(-200)
df[numeric_cols].head()

,prescription_dose_val_rx,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,pre_baseexcess,pre_totalco2,pre_hematocrit,pre_hemoglobin,pre_sodium,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,60.0,7.3,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
1,100.0,7.3,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2,50.0,7.3,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
3,250.0,7.3,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
4,325.0,7.3,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0


In [5]:
# Find the maximum value across all numeric values in the selected columns (ignoring NaN values)
max_value = df[numeric_cols].max().max()

# Now, divide only the numeric values in the selected columns by the max_value
df.loc[:, numeric_cols] = df[numeric_cols].div(max_value)

df.head()

,subject_id,prescription_start,prescription_drug,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,Phenylephrine HCl,0.0024,mg,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
1,10013,2125-10-05T00:00:00,Vasopressin,0.0040,UNIT,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
2,10013,2125-10-05T00:00:00,Iso-Osmotic Dextrose,0.0020,ml,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
3,10013,2125-10-05T00:00:00,Dobutamine,0.0100,mg,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
4,10013,2125-10-05T00:00:00,Aspirin,0.0130,mg,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008


In [6]:
# standardize units by converting to lowercase
df['prescription_dose_unit_rx'] = df['prescription_dose_unit_rx'].str.lower()

units_set = set(df['prescription_dose_unit_rx'])
# standardize units by converting to lowercase
df['prescription_dose_unit_rx'] = df['prescription_dose_unit_rx'].str.lower()

# # Standardize some redundant units
df['prescription_dose_unit_rx'] = df['prescription_dose_unit_rx'].replace({
    'g': 'gm',
    'mcg/h': 'mcg',
    'mcg/hr': 'mcg',
    'mcg/hour': 'mcg',
    'puff': 'unit',
    'syr': 'unit',
    'puff': 'unit',
    'pkt': 'unit',
    'tab': 'unit',
    'vial': 'unit',
    'bag': 'unit',
    'drop': 'unit',
    'inh': 'unit',
    'cap': 'unit',
})

units_set = set(df['prescription_dose_unit_rx'])

print(len(units_set))
print(units_set)

#Mapping for Ordinal Encoding
label_encoding = {}

mapping = 0

for elem in units_set:
    label_encoding[elem] = mapping
    mapping += 1

print(label_encoding)

15
{'unit', 'neb', 'mmol', 'ml', 'loz', 'appl', 'gm', 'enema', 'amp', 'mcg', 'mg', 'ptch', 'meq', 'troc', 'in'}
{'unit': 0, 'neb': 1, 'mmol': 2, 'ml': 3, 'loz': 4, 'appl': 5, 'gm': 6, 'enema': 7, 'amp': 8, 'mcg': 9, 'mg': 10, 'ptch': 11, 'meq': 12, 'troc': 13, 'in': 14}


In [7]:
df['prescription_dose_unit_rx'] = df['prescription_dose_unit_rx'].str.lower().map(label_encoding)

In [10]:
df.head()

,subject_id,prescription_start,prescription_drug,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,Phenylephrine HCl,0.0024,10,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
1,10013,2125-10-05T00:00:00,Vasopressin,0.0040,0,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
2,10013,2125-10-05T00:00:00,Iso-Osmotic Dextrose,0.0020,3,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
3,10013,2125-10-05T00:00:00,Dobutamine,0.0100,10,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
4,10013,2125-10-05T00:00:00,Aspirin,0.0130,10,2125-10-04T23:59:00,0.000292,0.00252,0.0024,-0.008,...,0.0024,0.009262,0.007667,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008,-0.008
